### SERPAPI 

In [4]:
import numpy as np
import pandas as pd

from datetime import datetime
from time import sleep

import pickle as pkl

from pprint import pprint
from libraries.serpapi import serpapi

In [5]:
# Cargamos los puestos y nuestra API_KEY
today = datetime.now().strftime("%Y-%m-%d")

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()

lista_puestos = pd.read_excel("data/lista_puestos.xlsx", header = None)
lista_puestos.columns = ["jobs"]
lista_puestos = lista_puestos["jobs"].tolist()
lista_puestos

['Desarrollador de software',
 ' Programador web',
 ' Consultor tecnológico',
 ' Ingeniero de sistemas',
 ' Especialista en seguridad informática',
 ' Analista programador',
 ' Desarrollador de aplicaciones móviles',
 ' Administrador de bases de datos',
 ' Arquitecto de software',
 ' Técnico de redes',
 ' Especialista en ciberseguridad',
 ' Project manager IT',
 ' Diseñador UX/UI',
 ' DevOps',
 ' Técnico de soporte',
 ' Analista de datos',
 ' Especialista en inteligencia artificial',
 ' Desarrollador de videojuegos',
 ' QA Tester',
 ' Consultor SAP',
 'Desarrollador full-stack',
 ' Diseñador gráfico digital',
 ' Especialista en blockchain',
 ' Científico de datos',
 ' Ingeniero de software',
 ' Consultor Big Data',
 ' Administrador de sistemas',
 ' Especialista en cloud computing',
 ' Analista de seguridad informática',
 ' Desarrollador Java',
 ' Técnico de sistemas',
 ' Especialista en machine learning',
 ' Desarrollador de Front-end',
 ' Especialista en IoT',
 ' Consultor de transfor

In [6]:
with open("data/lista_puestos_ingles.txt", "r") as file:
    lista_puestos_ingles = file.read()
    
lista_puestos_ingles = lista_puestos_ingles.split("\n")

lista_puestos_ingles = [x for x in [x.replace("Developer", "") for x in lista_puestos_ingles] if len(x.split()) > 1]

lista_puestos_ingles

['Web Programmer',
 'Technology Consultant',
 'Systems Engineer',
 'Information Security Specialist',
 'Programmer Analyst',
 'Mobile App ',
 'Database Administrator',
 'Software Architect',
 'Network Technician',
 'Cybersecurity Specialist',
 'IT Project Manager',
 'UX/UI Designer',
 'DevOps Engineer',
 'Support Technician',
 'Data Analyst',
 'Artificial Intelligence Specialist',
 'Video Game ',
 'QA Tester',
 'SAP Consultant',
 'Digital Graphic Designer',
 'Blockchain Specialist',
 'Data Scientist',
 'Software Engineer',
 'Big Data Consultant',
 'Systems Administrator',
 'Cloud Computing Specialist',
 'Information Security Analyst',
 'Systems Technician',
 'Machine Learning Specialist',
 'IoT Specialist (Internet of Things)',
 'Digital Transformation Consultant',
 'Virtual Reality Specialist',
 'Industrial Cybersecurity Specialist',
 'Telecommunications Technician',
 'Security Consultant',
 'Deep Learning Specialist',
 'Data Mining Specialist',
 'Network Specialist',
 'Enterprise Arc

In [7]:
# Ajustamos España como país de búsqueda
spain = [[".es", "Spain"]]

# Ajustamos Latino-américa como países de búsqueda
latam = [["com.ar", "Argentina"]         ,
         ["com.bo", "Bolivia"]           ,
         ["com.br", "Brazil"]            ,
         ["cl", "Chile"]             ,
         ["com.co", "Colombia"]          ,
         ["co.cr", "Costa Rica"]        ,
         ["com.cu", "Cuba"]              ,
         ["com.do", "Dominican Republic"],
         ["com.ec", "Ecuador"]           ,
         ["com.sv", "El Salvador"]       ,
         ["com.gt", "Guatemala"]         ,
         ["hn", "Honduras"]          ,
         ["com.mx", "Mexico"]            ,
         ["com.ni", "Nicaragua"]         ,
         ["com.pa", "Panama"]            ,
         ["com.py", "Paraguay"]          ,
         ["com.pe", "Peru"]              ,
         ["com.pr", "Puerto Rico"]       ,
         ["com.uy", "Uruguay"]]

In [8]:
%%time

df = pd.DataFrame()

for abr, country in spain:
    
    for q in lista_puestos: # Modificar lista de puestos según país/países de preferencia.

        try:
            
            for pagination in range(100):
                
                print(f"{q:60}{pagination}")

                q_params = {"q"             : q,
                            "api_key"       : api_key,
                            "location"      : country.title(),
                            "start"         : pagination*10,
                            "chips"         : "date_posted:3days" # Ofertas de empleo de hace 3 días hasta hoy
                            }

                response = serpapi.job_search(**q_params)


                if ("error" in response) or (response.get("jobs_results") == None) or (len(response.get("jobs_results")) < 10):
                    break

                df_response = pd.json_normalize(response["jobs_results"])

                df_response.columns = [x.split(".")[0] if len(x.split(".")) == 1 else x.split(".")[-1] for x in df_response.columns]

                df_response["country_search"] = country.title()
                
                df = pd.concat(objs = [df, df_response], ignore_index = True)
                
        except:
            
            print(f"Error {q} ***************************************************************")
                 
df = df.drop_duplicates(subset = "job_id").reset_index(drop = True) # Eliminamos duplicados si existen en "job_id"
df = df.drop_duplicates(subset = "description").reset_index(drop = True) # Eliminamos duplicados por descripción de oferta
df["date_posted"] = datetime.strptime(today, "%Y-%m-%d").date()

df.to_csv(f"data/extracted/extraction_{today}_spain.csv", index = False)

Desarrollador de software                                   0
Desarrollador de software                                   1
Desarrollador de software                                   2
Desarrollador de software                                   3
Desarrollador de software                                   4
Desarrollador de software                                   5
Desarrollador de software                                   6
Desarrollador de software                                   7
Desarrollador de software                                   8
Desarrollador de software                                   9
Desarrollador de software                                   10
 Programador web                                            0
 Programador web                                            1
 Programador web                                            2
 Programador web                                            3
 Programador web                                            4
 Consul

 Administrador de sistemas                                  1
 Administrador de sistemas                                  2
 Administrador de sistemas                                  3
 Administrador de sistemas                                  4
 Administrador de sistemas                                  5
 Especialista en cloud computing                            0
 Analista de seguridad informática                          0
 Desarrollador Java                                         0
 Desarrollador Java                                         1
 Desarrollador Java                                         2
 Desarrollador Java                                         3
 Desarrollador Java                                         4
 Desarrollador Java                                         5
 Desarrollador Java                                         6
 Desarrollador Java                                         7
 Técnico de sistemas                                        0
 Técnico

 Programador .NET                                           2
 Programador .NET                                           3
 Programador .NET                                           4
 Desarrollador de software Java                             0
 Desarrollador de software Java                             1
 Desarrollador de software Java                             2
 Desarrollador de software Java                             3
 Desarrollador de software Java                             4
 Desarrollador de software Java                             5
 Desarrollador de software Java                             6
 Especialista en Business Intelligence                      0
 Técnico de virtualización                                  0
 Desarrollador de software .NET                             0
 Desarrollador de software .NET                             1
 Desarrollador de software .NET                             2
 Desarrollador de software .NET                             3
 Especia

 Desarrollador de software Sharepoint                       0
 Especialista en redes Aruba                                0
 Especialista en seguridad de redes Cisco                   0
 Técnico de soporte SQL Server                              0
 Especialista en redes F5                                   0
 Especialista en redes Palo Alto                            0
 Técnico de soporte SAS                                     0
 Especialista en seguridad de la información PCI-DSS        0
 Técnico de soporte AWS                                     0
 Especialista en virtualización Citrix                      0
 Técnico de soporte Linux                                   0
 Técnico de soporte IBM                                     0
 Técnico de soporte SAP HANA                                0
 Desarrollador de backend                                   0
 Desarrollador de frontend                                  0
 Desarrollador de frontend                                  1
 Desarro

Técnico de soporte                                          6
Técnico de soporte                                          7
Ingeniero de sistemas                                       0
Ingeniero de sistemas                                       1
Ingeniero de sistemas                                       2
Ingeniero de sistemas                                       3
Ingeniero de sistemas                                       4
Ingeniero de sistemas                                       5
 Consultor de TI                                            0
 Diseñador gráfico                                          0
 Diseñador gráfico                                          1
 Diseñador gráfico                                          2
 Diseñador gráfico                                          3
 Diseñador gráfico                                          4
 Diseñador gráfico                                          5
 Desarrollador full stack                                   0
 Desarro

 Especialista en desarrollo web                             0
 Especialista en experiencia del usuario                    0
Analista de seguridad informática                           0
 Desarrollador de software de escritorio                    0
 Especialista en gestión de proyectos tecnológicos          0
 Técnico de soporte técnico IT                              0
 Técnico de soporte técnico IT                              1
 Técnico de soporte técnico IT                              2
 Técnico de soporte técnico IT                              3
 Técnico de soporte técnico IT                              4
 Técnico de soporte técnico IT                              5
 Desarrollador de software de sistemas                      0
 Desarrollador de software de sistemas                      1
 Desarrollador de software de sistemas                      2
 Desarrollador de software de sistemas                      3
 Desarrollador de software de sistemas                      4
 Desarro

 Consultor de seguridad IT                                  2
 Consultor de seguridad IT                                  3
 Consultor de seguridad IT                                  4
 Consultor de sistemas                                      0
 Consultor de sistemas                                      1
 Consultor de sistemas                                      2
 Consultor de sistemas                                      3
 Desarrollador de inteligencia artificial                   0
 Desarrollador de inteligencia artificial                   1
 Diseñador de interiores en Realidad Virtual                0
 Diseñador de sistemas                                      0
 Diseñador gráfico multimedia                               0
Programador de PLCs                                         0
Programador de PLCs                                         1
 Desarrollador de sistemas embebidos                        0
 Especialista en ciberinteligencia                          0
 Program

 Desarrollador React                                        3
 Desarrollador React                                        4
 Desarrollador React                                        5
 Especialista en seguridad de aplicaciones                  0
 Especialista en seguridad de aplicaciones                  1
 Ingeniero de sistemas embebidos                            0
Desarrollador Angular                                       0
Desarrollador Angular                                       1
Desarrollador Angular                                       2
Desarrollador Angular                                       3
Desarrollador Angular                                       4
 Analista de datos de recursos humanos                      0
 Programador COBOL                                          0
 Programador COBOL                                          1
 Especialista en sistemas de control de procesos            0
 Técnico de soporte de aplicaciones                         0
 Técnico

In [ ]:
################################################################################################################################